## Numerical Solution of LWR model:

Lighthill-Whitham-Richards model (commonly used for traffic flow):

\begin{equation}
\frac{\partial q}{\partial x} + \frac{\partial k}{\partial t} = 0
\end{equation}

But $q$ is a function of $k$ (Greenshield model): 
$$
q(k) = k(k_j - k) = k k_j -k^2
$$

Therefore
$$
\frac{\partial k}{\partial t} + \frac{dq}{dk}\frac{\partial k}{\partial x} =0
$$

$$
\frac{d(q(k))}{dk} = 1(k_j - k) + k(-1) = k_j - 2k
$$

And
$$
\frac{\partial k}{\partial t} + (k_j - 2k) \frac{\partial k}{\partial x} = 0
$$

$$
\frac{\partial k}{\partial t} = (2k - k_j) \frac{\partial k}{\partial x}
$$

\begin{align}
k(t + \Delta t, x_n) =& k(t, x) + \Delta t \frac{\partial k}{\partial t} \\
                =& k(t, x) + \Delta t \left(2k(t, x) - k_j\right) \frac{\partial k}{\partial x}
\end{align}

\begin{align}
\frac{\partial{k}}{\partial x} \approx \frac{k(t, x + \Delta x) - k(t,x - \Delta x)}{2 \Delta x} \\
                               \approx \frac{k(t, x + \Delta x) - k(t, x)}{\Delta x} \\
\end{align}

If I remember correctly the central difference formula is unstable when used with the forward time step, 
but I will check into this.

Discretizing along t_i for i=0,$n_t$ and x_n for n=0, $n_x$.
$k_n^i$ equals the traffic density at time $t_i$ and position $n_i$.

Sampling rates are $\Delta t$ and $\Delta x$, respectively.
$k_j(x)$ is location dependent.  Call it $\kappa_n$.

$$
k_n^{i+1} = k_n^i + \Delta t  (2 k_n^i - \kappa_n) \frac{k_{n+1}^i - k_n^i}{\Delta x}
$$

In [384]:
import numpy as np

class traffic_density:
    def __init__(self, nx, nt, dx, dt, kj):
        self.nx = nx
        self.nt = nt
        self.k=np.zeros((nt,nx), dtype=float) 
        #initial conditions
        self.k[0,:] = 0.
        self.kappa=kj
        self.dx = dx
        self.dt = dt
        
        #boundary conditions (traffic is at optimal density)
        self.k[0:50,0] = self.kappa[0] / 2.
    
    def forward(self):
        for i in range(1,self.nt-1):
            for n in range(1,self.nx):
                self.k[i+1,n] = self.k[i,n] + self.dt*(2*self.k[i,n] - self.kappa[n]) * \
                (self.k[i,n]-self.k[i,n-1])/self.dx
    
    def getq(self):
        self.q=np.zeros((self.nt,self.nx), dtype=float)
        for i in range(self.nt-1):
            self.q[i,:] = self.k[i,:] * (self.kappa - self.k[i,:])
            
    def getvel(self):
        self.u=np.zeros((self.nt, self.nx))
        for i in range(self.nt-1):
            idx= abs(self.k[i,:]) > 0
            self.u[i,idx] = self.q[i,idx]/self.k[i,idx]
            idx= self.k[i,:] == 0
            self.u[i,idx] = np.nan

In [385]:
nx=100

nt=1000
#kj = np.linspace(275/3600., 75/3600., nx)
kj = np.ones((nx,), dtype=float)*0.05
kj[50:60] = 0.01 #50/3600.

dx=0.1
dt=0.5
print(dt)

td=traffic_density(nx, nt, dx, dt, kj)
td.forward()
td.getq()
td.getvel()

0.5


In [386]:
#td.k[99,:]

In [387]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets

In [389]:
#widgets.IntSlider?

In [390]:
tslider=widgets.IntSlider(
    value=1.0, min=1.0, max=nt, step=1, description='Time Step',
    continuous_update=False)

ti=0
trace1=go.Scatter(y=td.k[ti,:], mode='lines', name='k')
trace2=go.Scatter(y=td.q[ti,:], mode='lines', name='q')
trace3=go.Scatter(y=td.u[ti,:], mode='lines', name='u')

fig = make_subplots(rows=3, cols=1, subplot_titles=["Density","Flow","Velocity"],  shared_xaxes=True)

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=2, col=1)
fig.add_trace(trace3, row=3, col=1)

fig.update_yaxes(range=[0, td.k.max()], row=1, col=1)
fig.update_yaxes(range=[0, td.q.max()], row=2, col=1)
fig.update_layout(height=600, width=800)
#def update_fig():
    #fig.add_trace(go.Scatter(y=td.k[ti,:], mode='lines', name='k'), row=1, col=1)
    #fig.add_trace(go.Scatter(y=td.q[ti,:], mode='lines', name='q'), row=2, col=1)
    #fig.add_trace(go.Scatter(y=td.u[ti,:], mode='lines', name='u'), row=3, col=1)

#fig.show()
fo=go.FigureWidget(fig)
#fig.add_trace(go.Scatter(y=td.k[300,:], mode='lines', name='k'), row=1, col=1)
def update_traces(value):
    ti=value["new"]
    fo.data[0].y=td.k[ti,:]
    fo.data[1].y=td.q[ti,:]
    fo.data[2].y=td.u[ti,:]

tslider.observe(update_traces, names='value')

#fo.data[0].y=td.k[300,:]
display(tslider)
display(fo)

IntSlider(value=1, continuous_update=False, description='Time Step', max=1000, min=1)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'k',
              'type': 'scatter',
    …

In [ ]:
fig.get_subplot